# CHANGE VECTOR ANALYSIS IN POSTERIOR PROBABILITY SPACE

In [1]:
import ee
import json
import os
import geemap

from dotenv import load_dotenv
from utils2 import (create_composite_DW, create_composite_S2, 
                    threshold_optimization, change_type_discrimination,
                    remove_small_objects, create_index_mask, dilate_mask,
                    erode_mask)

## Initialize Google Earth Engine API

In [2]:
NOTEBOOK_DIR = os.getcwd()

# Get the project root directory (two levels up)
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, '../..'))

# Load environment variables
load_dotenv()

# Get credentials path and ensure it's relative to PROJECT_ROOT
GEE_CREDENTIALS_PATH = os.path.join(PROJECT_ROOT, os.getenv('GEE_CREDENTIALS_PATH'))
GEE_PROJECT_ID = os.getenv('GEE_PROJECT_ID')

# Initialize GEE
credentials = ee.ServiceAccountCredentials(
    '',
    GEE_CREDENTIALS_PATH,
    GEE_PROJECT_ID
)
ee.Initialize(credentials, opt_url="https://earthengine-highvolume.googleapis.com")

## Use GEE Sentinel-2 image with respective Dynamic Land Cover Map

### Input required
- T1: Sentinel-2 image in time 1
- T2: Sentinel-2 image in time 2
- coordinates: coordinates of the area of interest

In [125]:
T1_date = '2021-01-01'
T2_date = '2022-12-31'
range_days = 30
coordinates = [
          [
            [
              -63.406465892673396,
              -30.347535161917683
            ],
            [
              -63.425223128637995,
              -30.37080128979506
            ],
            [
              -63.42803671537233,
              -30.397095465965478
            ],
            [
              -63.400509336566884,
              -30.42050193711671
            ],
            [
              -63.37073222447286,
              -30.40432562167593
            ],
            [
              -63.34916140311282,
              -30.391787133563383
            ],
            [
              -63.35971234834342,
              -30.36245696700476
            ],
            [
              -63.381752100602284,
              -30.345259330244637
            ],
            [
              -63.39769575117229,
              -30.339391209687783
            ],
            [
              -63.406465892673396,
              -30.347535161917683
            ]
          ]
        ]

### Pre-processing pipeline using GEE
- Atmospheric correction
- Cloud & Shadow masking
- Image registration
- BRDF correction

In [126]:
# --- Preprocessing team should be able to provide the following ---
## Region of interest defined by a polygon
roi = ee.Geometry.Polygon(coordinates)

## Create a composite Sentinel-2 image for time T1 and clip it to the ROI
S2_T1 = create_composite_S2(T1_date, range_days, roi).clip(roi)

## Create a composite Sentinel-2 image for time T2 and clip it to the ROI
S2_T2 = create_composite_S2(T2_date, range_days, roi).clip(roi)

## Create spectral index masks by a threshold
dndvi_mask = create_index_mask(S2_T1, S2_T2, index='dNDVI', threshold=-0.2)
dnbr_mask = create_index_mask(S2_T1, S2_T2, index='dNBR', threshold=-0.3)
dsavi_mask = create_index_mask(S2_T1, S2_T2, index='dSAVI', threshold=-0.3)
dndmi_mask = create_index_mask(S2_T1, S2_T2, index='dNDMI', threshold=-0.3)

Sentinel-2 available images: 6
Sentinel-2 available images: 5


### Change vector analysis in posterior probability space
- Calculate posterior probability of change (using Dynamic Land Cover Probability Map)
- Calculate change vector in posterior probability space
- Calculate magnitude and direction of change vector
- Discriminate between different types of change (get only the significant changes between tree cover and non-tree cover)
- Apply an iterative thresholding method to get the final change map

In [127]:
# --- Change detection using CVA technique (model team) ---
## Create composites for each date using the create_composite_DW function
composite_T1 = create_composite_DW(T1_date, range_days, roi).clip(roi).updateMask(S2_T1.select('B4').mask())
composite_T2 = create_composite_DW(T2_date, range_days, roi).clip(roi).updateMask(S2_T2.select('B4').mask())

## Compute the change vector between the composites
delta_prob = composite_T2.subtract(composite_T1)

## Calculate the magnitude of change using the Euclidean norm across bands
magnitude = delta_prob.pow(2).reduce(ee.Reducer.sum()).sqrt()

## Convert the image to an array and get the index of the maximum value along axis 0 (bands)
labels_t1 = composite_T1.toArray().arrayArgmax().arrayGet([0]).rename('labels_t1')
labels_t2 = composite_T2.toArray().arrayArgmax().arrayGet([0]).rename('labels_t2')

## Optimize the threshold value
step_coarse, step_fine = 0.2, 0.1
best_threshold, best_Lk = threshold_optimization(magnitude, labels_t1, labels_t2, 
                                                 roi, 10, step_coarse, step_fine)
print("Best threshold:", best_threshold, "with Lk =", best_Lk)

## Generate the change mask by applying the optimized threshold
changed_mask = magnitude.gte(ee.Image.constant(best_threshold))

## Apply change type discrimination
## It is assumed that change_type_discrimination returns an ee.Image with the transition code (a*100 + b) for each pixel.
change_map = change_type_discrimination(composite_T1, composite_T2, changed_mask, 9)

## It is assumed that the forest class is 1, so transitions with a = 1 will have codes between 100 and 200.
forest_change = change_map.gte(100).And(change_map.lte(200))

Dynamic World available images: 5
Dynamic World available images: 5
Best threshold: 0.20506942708597878 with Lk = 53.742271945436364


### Post-processing pipeline using GEE
- Improve the change map using spectral indices masking
- Use morphological operations to remove noise (remove small objects, dilate, erode)
- Get the final change map as a vector and export it as a GeoJSON file

In [128]:
# --- Post-processing (remove small objects, dilate, and vectorize) ---
## Adjust the forest change image by multiplying it with each of the spectral index masks
# Sum the spectral masks (each mask is assumed to have 1 for deforestation and 0 for no deforestation)
spectral_vote = forest_change.add(dndvi_mask).add(dnbr_mask).add(dndmi_mask).add(dsavi_mask)

# Create the majority vote mask: at least 3 out of 5 masks must indicate deforestation
majority_vote_mask = spectral_vote.gte(3)

## Dilate and then erode the change mask to smooth the edges
forest_change_dilated = dilate_mask(majority_vote_mask, radius=1)
forest_change_eroded  = erode_mask(forest_change_dilated, radius=1)

## Remove small objects using the remove_small_objects function
forest_change_cleaned = remove_small_objects(forest_change_eroded, min_size=16, connectivity=8)

## Vectorize the cleaned image to obtain polygons of forest change
forest_change_vectors = forest_change_cleaned.updateMask(forest_change_cleaned).reduceToVectors(
    geometry=roi,
    scale=30, 
    geometryType='polygon',
    reducer=ee.Reducer.countEvery()
)

### Visualization of the results in GEE map

In [129]:
# Create a new geemap interactive map
Map = geemap.Map(basemap='SATELLITE')

# Center the map on the region of interest (roi) with zoom level 10
Map.centerObject(roi, 13)

# Add the T1 composite image to the map with a natural-color visualization (RGB using bands B4, B3, B2)
Map.addLayer(
    S2_T1, 
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 
    'S2 T1'
)

# Add the T2 composite image to the map with the same visualization parameters
Map.addLayer(
    S2_T2, 
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 
    'S2 T2'
)

# Add the DW composites to the map
Map.addLayer(
    composite_T1,
    {'bands': ["trees"], 'min': 0, 'max': 1, 'palette': ['black', 'purple']},
    'DW Composite T1'
)

Map.addLayer(
    composite_T2,
    {'bands': ["trees"], 'min': 0, 'max': 1, 'palette': ['black', 'purple']},
    'DW Composite T2'
)

# Add the forest change map and the spectral index masks to the map
Map.addLayer(
    forest_change,
    {'min': 0, 'max': 1, 'palette': ['black', 'white']},
    'Forest Change Model Mask'
)

Map.addLayer(  
    dndvi_mask,
    {'min': 0, 'max': 1, 'palette': ['black', 'red']},
    'dNDVI Mask'
)

Map.addLayer(
    dsavi_mask,
    {'min': 0, 'max': 1, 'palette': ['black', 'yellow']},
    'dSAVI Mask'
)

Map.addLayer(
    dnbr_mask,
    {'min': 0, 'max': 1, 'palette': ['black', 'blue']},
    'dNBR Mask'
)

Map.addLayer(
    dndmi_mask,
    {'min': 0, 'max': 1, 'palette': ['black', 'green']},
    'dNDMI Mask'
)

# Add the final forest change vectors to the map
Map.addLayer(
    ee.Image().paint(forest_change_vectors, 0, 2),
    {'palette': 'FF0000'},
    'Forest Change Vectors'
)

# Add the region of interest (ROI) as a blue outline for reference.
Map.addLayer(
    ee.Image().paint(roi, 0, 2), 
    {'palette': 'blue'}, 
    'ROI'
)

# Display the map
Map

Map(center=[-30.380401346656825, -63.39125021628172], controls=(WidgetControl(options=['position', 'transparen…

### Bibliography

```
@ARTICLE{5597922,
  author={Chen, Jin and Chen, Xuehong and Cui, Xihong and Chen, Jun},
  journal={IEEE Geoscience and Remote Sensing Letters}, 
  title={Change Vector Analysis in Posterior Probability Space: A New Method for Land Cover Change Detection}, 
  year={2011},
  volume={8},
  number={2},
  pages={317-321},
  keywords={Remote sensing;Pixel;Radiometry;Accuracy;Training;Satellites;Earth;Change vector analysis (CVA);land cover change;postclassification comparison (PCC);posterior probability space},
  doi={10.1109/LGRS.2010.2068537}}

```